# Imports

In [1]:
import pandas as pd
import numpy as np

import torch.nn as nn
import torch

import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer

import random
import os
from more_itertools import flatten

In [2]:
test=pd.read_csv("../input/commonlitreadabilityprize/test.csv")
test["target"]=0

In [3]:
MAX_LEN = 256
ROBERTA_LARGE_PATH = "../input/robertalarge"
ROBERTA_BASE_PATH = "../input/roberta-new"
ELECTRA_PATH = '../input/electralarge'
TEST_BATCH_SIZE = 8
TOKENIZER_ROBERTA = transformers.AutoTokenizer.from_pretrained(ROBERTA_LARGE_PATH)
TOKENIZER_ELECTRA = AutoTokenizer.from_pretrained(ELECTRA_PATH)

# Datasets

In [4]:
class RobertaDataset:
    def __init__(self,df):
        self.excerpt = df.excerpt.values
        self.target = df.target.values

    def __len__(self):
        return len(self.excerpt)
    
    def __getitem__(self,item):
        excerpt = str(self.excerpt[item])
        excerpt = " ".join(excerpt.split())
        inputs = TOKENIZER_ROBERTA(excerpt, add_special_tokens=True, max_length=MAX_LEN, padding=True, truncation=True)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        
        padding_len = MAX_LEN-len(ids)
        ids = ids+([0]*padding_len)
        mask = mask+([0]*padding_len)
 
        return {"ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float)}

In [5]:
class ElectraDataset:
    def __init__(self,df):
        self.excerpt = df.excerpt.values
        self.target = df.target.values

    def __len__(self):
        return len(self.excerpt)
    
    def __getitem__(self,item):
        excerpt = str(self.excerpt[item])
        excerpt = " ".join(excerpt.split())
        inputs = TOKENIZER_ELECTRA(excerpt, add_special_tokens=True, max_length=MAX_LEN, padding=True, truncation=True)
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        
        padding_len = MAX_LEN-len(ids)
        ids = ids+([0]*padding_len)
        mask = mask+([0]*padding_len)
        token_type_ids = token_type_ids+([0]*padding_len)
 
        return {"ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets": torch.tensor(self.target[item], dtype=torch.float)}

# Models

In [6]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_BASE_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_BASE_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, ids, mask, loss_fn = None, targets = None):
        roberta_output = self.roberta(input_ids=ids,
                                      attention_mask=mask)        

        last_layer_hidden_states = roberta_output.hidden_states[-1]

        weights = self.attention(last_layer_hidden_states)
                
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        return self.regressor(context_vector)

In [7]:
class RobertaBaseAttention(nn.Module):
    
    def __init__(self, model_type="attention"):
        super(RobertaBaseAttention,self).__init__()

        self.model_type = model_type
        
        self.config = AutoConfig.from_pretrained(ROBERTA_BASE_PATH)
        self.config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})   
        self.roberta = transformers.AutoModel.from_pretrained(ROBERTA_BASE_PATH, config=self.config)

        if model_type == "attention":
            
            self.attention = nn.Sequential(            
            nn.Linear(768, 256),            
            nn.Tanh(),                       
            nn.Linear(256, 1),
            nn.Softmax(dim=1)
            )   

            self.layer_norm1 = nn.LayerNorm(768)
            self.linear1 = nn.Linear(768, 256)
            self.linear2 = nn.Linear(256, 1)
            self.layer_norm2 = nn.LayerNorm(256)
                           
        elif model_type == "mean":
        
            self.layer_norm1 = nn.LayerNorm(1024)
            self.linear1 = nn.Linear(1024, 256)
            self.linear2 = nn.Linear(256, 1)
            self.layer_norm2 = nn.LayerNorm(256)

    def freeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = False

    def unfreeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = True
        
    def forward(self, ids, mask, loss_fn = None, targets = None):

        if self.model_type == "mean":

            outputs = self.roberta(ids, mask)
            last_hidden_state = outputs[0]
            input_mask_expanded = mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            norm_mean_embeddings = self.layer_norm1(mean_embeddings)
            logits = self.linear1(norm_mean_embeddings)
            logits = self.linear2(self.layer_norm2(logits))

        elif self.model_type=="attention":

            roberta_output = self.roberta(input_ids=ids,
                                  attention_mask=mask)        
            last_layer_hidden_states = roberta_output.last_hidden_state
            weights = self.attention(last_layer_hidden_states)
            context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)
            norm_context_vector = self.layer_norm1(context_vector)
            logits = self.linear1(norm_context_vector)
            logits = self.linear2(self.layer_norm2(logits)) 

        return logits

In [8]:
class RobertaLargeAttention(nn.Module):
    
    def __init__(self, model_type="attention"):
        super(RobertaLargeAttention,self).__init__()

        self.model_type = model_type
        
        self.config = AutoConfig.from_pretrained(ROBERTA_LARGE_PATH)
        self.config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})   
        self.roberta = transformers.AutoModel.from_pretrained(ROBERTA_LARGE_PATH, config=self.config)

        if model_type == "attention":
            
            self.attention = nn.Sequential(            
            nn.Linear(1024, 256),            
            nn.Tanh(),                       
            nn.Linear(256, 1),
            nn.Softmax(dim=1)
            )   

            self.layer_norm1 = nn.LayerNorm(1024)
            self.linear1 = nn.Linear(1024, 256)
            self.linear2 = nn.Linear(256, 1)
            self.layer_norm2 = nn.LayerNorm(256)
                           
        elif model_type == "mean":
        
            self.layer_norm1 = nn.LayerNorm(1024)
            self.linear1 = nn.Linear(1024, 256)
            self.linear2 = nn.Linear(256, 1)
            self.layer_norm2 = nn.LayerNorm(256)

    def freeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = False

    def unfreeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = True
        
    def forward(self, ids, mask, loss_fn = None, targets = None):

        if self.model_type == "mean":

            outputs = self.roberta(ids, mask)
            last_hidden_state = outputs[0]
            input_mask_expanded = mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            norm_mean_embeddings = self.layer_norm1(mean_embeddings)
            logits = self.linear1(norm_mean_embeddings)
            logits = self.linear2(self.layer_norm2(logits))

        elif self.model_type=="attention":

            roberta_output = self.roberta(input_ids=ids,
                                  attention_mask=mask)        
            last_layer_hidden_states = roberta_output.last_hidden_state
            weights = self.attention(last_layer_hidden_states)
            context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)
            norm_context_vector = self.layer_norm1(context_vector)
            logits = self.linear1(norm_context_vector)
            logits = self.linear2(self.layer_norm2(logits)) 

        return logits

In [9]:
# path : Models/CodeRobertaLargeMean/model{fold}.bin
class RobertaLargeMean(nn.Module):
    
    def __init__(self, model_type="mean"):
        super(RobertaLargeMean,self).__init__()

        self.model_type = model_type
        
        self.config = AutoConfig.from_pretrained(ROBERTA_LARGE_PATH)
        self.config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})   
        self.roberta = transformers.AutoModel.from_pretrained(ROBERTA_LARGE_PATH, config=self.config)

        if model_type == "attention":
            
            self.attention = nn.Sequential(            
            nn.Linear(1024, 256),            
            nn.Tanh(),                       
            nn.Linear(256, 1),
            nn.Softmax(dim=1)
            )   

            self.linear = (nn.Linear(1024, 1))
                           
        elif model_type == "mean":
        
            self.layer_norm1 = nn.LayerNorm(1024)
            self.linear1 = nn.Linear(1024, 768)
            self.linear2 = nn.Linear(768, 1)
            self.layer_norm2 = nn.LayerNorm(768)

    def freeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = False

    def unfreeze(self):
        for child in self.roberta.children():
            for param in child.parameters():
                param.requires_grad = True
        
    def forward(self, ids, mask, loss_fn = None, targets = None):

        if self.model_type == "mean":

            outputs = self.roberta(ids, mask)
            last_hidden_state = outputs[0]
            input_mask_expanded = mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            norm_mean_embeddings = self.layer_norm1(mean_embeddings)
            logits = self.linear1(norm_mean_embeddings)
            logits = self.linear2(self.layer_norm2(logits))

        elif self.model_type=="attention":

            roberta_output = self.roberta(input_ids=ids,
                                  attention_mask=mask)        
            last_layer_hidden_states = roberta_output.last_hidden_state
            weights = self.attention(last_layer_hidden_states)
            context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
            logits = self.linear(context_vector)

        return logits

In [10]:
# Attention for roberta path : Models/CodeElectraLargeBaseline/model{fold}.bin
class ElectraLarge(nn.Module):
    
    def __init__(self, model_type="mean"):
        super(ElectraLarge,self).__init__()

        self.model_type = model_type
        
        self.config = AutoConfig.from_pretrained(ELECTRA_PATH)
        self.config.update({ 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7
                       })   
        
        self.electra = AutoModel.from_pretrained(ELECTRA_PATH, config=self.config)

        if model_type == "attention":
            
            self.attention = nn.Sequential(            
            nn.Linear(1024, 256),            
            nn.Tanh(),                       
            nn.Linear(256, 1),
            nn.Softmax(dim=1)
            )   

            self.linear = (nn.Linear(1024, 1))
                           
        elif model_type == "mean":
        
            self.layer_norm1 = nn.LayerNorm(1024)
            self.linear1 = nn.Linear(1024, 768)
            self.linear2 = nn.Linear(768, 1)
            self.layer_norm2 = nn.LayerNorm(768)
        
    def forward(self, ids, mask, token_type_ids, loss_fn = None, targets = None):

        if self.model_type == "mean":

            outputs = self.electra(input_ids=ids, attention_mask=mask, token_type_ids = token_type_ids)
            last_hidden_state = outputs.last_hidden_state
            input_mask_expanded = mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
            sum_mask = input_mask_expanded.sum(1)
            sum_mask = torch.clamp(sum_mask, min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            norm_mean_embeddings = self.layer_norm1(mean_embeddings)
            logits = self.linear1(norm_mean_embeddings)
            logits = self.linear2(self.layer_norm2(logits))

        return logits

# Create Predictions

In [11]:
def inf_fn(data_loader_electra,data_loader_roberta, model, device):

    model.eval()

    if "Electra" in model.__class__.__name__ :

        with torch.no_grad():
            for index, d in enumerate(data_loader_electra):
                ids = d["ids"]
                mask = d["mask"]
                token_type_ids = d["token_type_ids"]

                ids = ids.to(device, dtype=torch.long)
                mask = mask.to(device, dtype=torch.long)
                token_type_ids = token_type_ids.to(device, dtype=torch.long)

                outputs = model(ids = ids, mask = mask, token_type_ids = token_type_ids)
                outputs = outputs.cpu().detach().numpy()

                if index == 0:
                    preds_test = outputs
                else:
                    preds_test = np.concatenate((preds_test,outputs), axis=None)

    else:

        with torch.no_grad():
                for index, d in enumerate(data_loader_roberta):
                    ids = d["ids"]
                    mask = d["mask"]

                    ids = ids.to(device, dtype=torch.long)
                    mask = mask.to(device, dtype=torch.long)

                    outputs = model(ids=ids, mask=mask)

                    outputs = outputs.cpu().detach().numpy()

                    if index == 0:
                        preds_test = outputs
                    else:
                        preds_test = np.concatenate((preds_test,outputs), axis=None)
          
    return preds_test

# Useful Functions

In [12]:
# create pytorch dataloader
def create_dataloader():

    valid_electra_dataset = ElectraDataset(test)
    valid_electra_dataloader = torch.utils.data.DataLoader(valid_electra_dataset, batch_size= TEST_BATCH_SIZE)
    valid_roberta_dataset = RobertaDataset(test)
    valid_roberta_dataloader = torch.utils.data.DataLoader(valid_roberta_dataset, batch_size= TEST_BATCH_SIZE)

    return valid_electra_dataloader, valid_roberta_dataloader

In [13]:
def create_model(device, name, path):
    
    if name == "coderobertalargeattentionnorm2":

        model = RobertaLargeAttention().to(device)
        model.load_state_dict(torch.load(path))

    elif name == "coderobertalargemean":

        model = RobertaLargeMean().to(device)
        model.load_state_dict(torch.load(path))

    elif name == "electra":

        model = ElectraLarge().to(device)
        model.load_state_dict(torch.load(path))

    elif name == "coderobertabaseattentionnorm":

        model = RobertaBaseAttention().to(device)
        model.load_state_dict(torch.load(path))
    

    else:
        raise Exception(f"Unknown model: {name}")
    
    return model

In [14]:
def make_predictions(test, name, path, valid_electra_dataloader, valid_roberta_dataloader ):
    device = torch.device("cuda")
    model = create_model(device, name, path)
    model.load_state_dict(torch.load(path))

    results = inf_fn(valid_electra_dataloader,valid_roberta_dataloader , model, device)

    return results

# Create Submission file

In [15]:
valid_electra_dataloader, valid_roberta_dataloader = create_dataloader()

In [16]:
dict_blend = {'electra': 0.3, 'coderobertalargemean': 0.3, 'coderobertalargeattentionnorm2': 0.2, 'coderobertabaseattentionnorm': 0.2}

In [17]:
models = ["coderobertabaseattentionnorm", "electra", "coderobertalargemean", "coderobertalargeattentionnorm2"]
predictions = []
for fold in range(5):
    for model in models:
        print(f"Inference for fold {fold} for model {model}")
            
        preds = make_predictions(test,model, f"../input/{model}/model{fold}.bin", valid_electra_dataloader, valid_roberta_dataloader)
        predictions.append(dict_blend[model] * preds/5)
        

Inference for fold 0 for model coderobertabaseattentionnorm
Inference for fold 0 for model electra


Some weights of the model checkpoint at ../input/electralarge were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Inference for fold 0 for model coderobertalargemean


Some weights of the model checkpoint at ../input/robertalarge were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Inference for fold 0 for model coderobertalargeattentionnorm2


...

Some weights of the model checkpoint at ../input/electralarge were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Inference for fold 4 for model coderobertalargemean


Some weights of the model checkpoint at ../input/robertalarge were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Inference for fold 4 for model coderobertalargeattentionnorm2


Some weights of the model checkpoint at ../input/robertalarge were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# dict_blend = {'codeelectralargebaseline': 0.3, 'coderobertalargemean': 0.3, 'coderobertalargeattentionnorm2': 0.2, 'coderobertabaseattentionnorm': 0.2}
# models = ["codeelectralargebaseline", "coderobertalargemean", "coderobertalargeattentionnorm2", "coderobertabaseattentionnorm"]
# predictions = []
# for fold in range(5):
#     for model in models:
#         print(f"Inference for fold {fold} for model {model}")
#         preds = make_predictions(test,model, f"../input/{model}/model{fold}.bin", valid_electra_dataloader, valid_roberta_dataloader)
#         predictions.append(dict_blend[model] * preds/5)
        

In [19]:
a = np.sum(predictions, axis=0)

test.target=a
test=test[["id","target"]]
test.to_csv("submission.csv",index=False)
test.head(10)

,id,target
0,c0f722661,-0.418750
1,f0953f0a5,-0.354951
2,0df072751,-0.430480
3,04caf4e0c,-2.353032
4,0e63f8bea,-1.903276
5,12537fe78,-1.224088
6,965e592c0,0.276594


In [20]:
# 23h30